# Projet Final - Analyse des Factures (Invoice Dataset)

**Dataset**: invoices.csv

**Membres de l'équipe**:
- Prénom NOM (Responsable)
- Prénom NOM (Membre)
- Prénom NOM (Membre)
- Prénom NOM (Membre)

**Objectif du projet**: Analyser les données de facturation d'une boutique en ligne pour extraire 4 indicateurs clés permettant de comprendre les tendances temporelles des ventes, la distribution géographique des clients, les produits les plus rentables et les patterns de consommation.

## Importation des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Définition des fonctions

In [2]:
def charger_donnees(chemin_fichier: str) -> pd.DataFrame:
    """
    Charge les données depuis un fichier CSV.
    
    Args:
        chemin_fichier: Chemin vers le fichier CSV
        
    Returns:
        DataFrame pandas contenant les données
    """
    df = pd.read_csv(chemin_fichier)
    return df

In [3]:
def explorer_donnees(df: pd.DataFrame) -> dict:
    """
    Explore et résume les caractéristiques principales du dataset.
    
    Args:
        df: DataFrame à explorer
        
    Returns:
        Dictionnaire contenant les statistiques descriptives
    """
    exploration = {
        'dimensions': df.shape,
        'colonnes': df.columns.tolist(),
        'types': df.dtypes.to_dict(),
        'valeurs_manquantes': df.isnull().sum().to_dict(),
        'statistiques': df.describe().to_dict()
    }
    return exploration

In [4]:
def preparer_donnees(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prépare les données en convertissant les dates et en créant des colonnes dérivées.
    
    Args:
        df: DataFrame brut
        
    Returns:
        DataFrame préparé avec colonnes supplémentaires
    """
    df_prep = df.copy()
    df_prep['invoice_date'] = pd.to_datetime(df_prep['invoice_date'], format='%d/%m/%Y')
    df_prep['annee'] = df_prep['invoice_date'].dt.year
    df_prep['mois'] = df_prep['invoice_date'].dt.month
    df_prep['trimestre'] = df_prep['invoice_date'].dt.quarter
    df_prep['jour_semaine'] = df_prep['invoice_date'].dt.dayofweek
    df_prep['revenu_total'] = df_prep['qty'] * df_prep['amount']
    return df_prep

In [5]:
def analyser_top_produits(df: pd.DataFrame, n: int = 10) -> pd.DataFrame:
    """
    INDICATEUR 1 - Requête de groupement:
    Calcule le revenu total par produit et retourne les n produits les plus rentables.
    
    Args:
        df: DataFrame préparé
        n: Nombre de produits à retourner
        
    Returns:
        DataFrame avec les top produits et leurs statistiques
    """
    resultats = df.groupby('product_id').agg({
        'revenu_total': 'sum',
        'qty': 'sum',
        'invoice_date': 'count'
    }).rename(columns={'invoice_date': 'nombre_transactions'})
    
    resultats['prix_moyen'] = resultats['revenu_total'] / resultats['qty']
    resultats = resultats.sort_values('revenu_total', ascending=False).head(n)
    resultats = resultats.reset_index()
    
    return resultats

In [6]:
def normaliser_montants(df: pd.DataFrame, methode: str = 'minmax') -> pd.DataFrame:
    """
    INDICATEUR 2 - Transformation des données:
    Normalise les montants des transactions et les quantités.
    
    Args:
        df: DataFrame préparé
        methode: Méthode de normalisation ('minmax' ou 'zscore')
        
    Returns:
        DataFrame avec colonnes normalisées ajoutées
    """
    df_norm = df.copy()
    
    if methode == 'minmax':
        df_norm['montant_normalise'] = (
            (df_norm['amount'] - df_norm['amount'].min()) / 
            (df_norm['amount'].max() - df_norm['amount'].min())
        )
        df_norm['quantite_normalisee'] = (
            (df_norm['qty'] - df_norm['qty'].min()) / 
            (df_norm['qty'].max() - df_norm['qty'].min())
        )
    elif methode == 'zscore':
        df_norm['montant_normalise'] = (
            (df_norm['amount'] - df_norm['amount'].mean()) / df_norm['amount'].std()
        )
        df_norm['quantite_normalisee'] = (
            (df_norm['qty'] - df_norm['qty'].mean()) / df_norm['qty'].std()
        )
    
    df_norm['categorie_montant'] = pd.cut(
        df_norm['amount'],
        bins=[0, 25, 50, 75, 100],
        labels=['Faible', 'Moyen', 'Élevé', 'Très élevé'],
        include_lowest=True
    )
    
    return df_norm

In [7]:
def analyser_tendances_temporelles(df: pd.DataFrame) -> pd.DataFrame:
    """
    INDICATEUR 3 - Analyse temporelle:
    Analyse l'évolution des ventes dans le temps avec statistiques par période.
    
    Args:
        df: DataFrame préparé
        
    Returns:
        DataFrame avec les tendances temporelles agrégées
    """
    ventes_mensuelles = df.groupby([df['invoice_date'].dt.to_period('M')]).agg({
        'revenu_total': 'sum',
        'qty': 'sum',
        'product_id': 'count'
    }).rename(columns={'product_id': 'nombre_transactions'})
    
    ventes_mensuelles.index = ventes_mensuelles.index.to_timestamp()
    ventes_mensuelles = ventes_mensuelles.sort_index()
    
    ventes_mensuelles['revenu_moyen_mobile'] = (
        ventes_mensuelles['revenu_total'].rolling(window=3, min_periods=1).mean()
    )
    
    return ventes_mensuelles

In [8]:
def prevoir_ventes(df: pd.DataFrame, periodes_futures: int = 12) -> pd.DataFrame:
    """
    INDICATEUR 3 (suite) - Prévision temporelle:
    Utilise une moyenne mobile pour prévoir les ventes futures.
    
    Args:
        df: DataFrame avec tendances temporelles
        periodes_futures: Nombre de périodes à prévoir
        
    Returns:
        DataFrame avec prévisions
    """
    derniere_date = df.index[-1]
    moyenne_recente = df['revenu_total'].tail(6).mean()
    
    dates_futures = pd.date_range(
        start=derniere_date + pd.DateOffset(months=1),
        periods=periodes_futures,
        freq='MS'
    )
    
    previsions = pd.DataFrame({
        'date': dates_futures,
        'prevision': moyenne_recente
    })
    
    return previsions

In [9]:
def analyser_distribution_spatiale(df: pd.DataFrame) -> pd.DataFrame:
    """
    INDICATEUR 4 - Analyse spatiale:
    Analyse la distribution géographique des ventes par ville.
    
    Args:
        df: DataFrame préparé
        
    Returns:
        DataFrame avec statistiques par ville
    """
    distribution = df.groupby('city').agg({
        'revenu_total': ['sum', 'mean', 'count'],
        'qty': 'sum',
        'amount': 'mean'
    })
    
    distribution.columns = ['_'.join(col).strip() for col in distribution.columns.values]
    distribution = distribution.rename(columns={
        'revenu_total_sum': 'revenu_total',
        'revenu_total_mean': 'revenu_moyen',
        'revenu_total_count': 'nombre_transactions',
        'qty_sum': 'quantite_totale',
        'amount_mean': 'montant_moyen'
    })
    
    distribution = distribution.sort_values('revenu_total', ascending=False).head(20)
    distribution = distribution.reset_index()
    
    return distribution

In [10]:
def creer_graphique_top_produits(top_produits: pd.DataFrame) -> go.Figure:
    """
    Crée un graphique interactif pour les top produits.
    
    Args:
        top_produits: DataFrame des meilleurs produits
        
    Returns:
        Figure Plotly
    """
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=top_produits['product_id'].astype(str),
        y=top_produits['revenu_total'],
        marker_color='rgb(55, 83, 109)',
        text=top_produits['revenu_total'].round(2),
        textposition='auto',
        name='Revenu Total'
    ))
    
    fig.update_layout(
        title='Top 10 Produits par Revenu Total',
        xaxis_title='ID Produit',
        yaxis_title='Revenu Total ($)',
        template='plotly_white',
        height=400
    )
    
    return fig

In [11]:
def creer_graphique_normalisation(df_norm: pd.DataFrame) -> go.Figure:
    """
    Crée un graphique de distribution des catégories de montants.
    
    Args:
        df_norm: DataFrame avec données normalisées
        
    Returns:
        Figure Plotly
    """
    distribution_categories = df_norm['categorie_montant'].value_counts()
    
    fig = go.Figure(data=[
        go.Pie(
            labels=distribution_categories.index,
            values=distribution_categories.values,
            hole=0.3,
            marker=dict(colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
        )
    ])
    
    fig.update_layout(
        title='Distribution des Catégories de Montants (après discrétisation)',
        template='plotly_white',
        height=400
    )
    
    return fig

In [12]:
def creer_graphique_tendances(ventes_mensuelles: pd.DataFrame, previsions: pd.DataFrame) -> go.Figure:
    """
    Crée un graphique des tendances temporelles avec prévisions.
    
    Args:
        ventes_mensuelles: DataFrame des ventes mensuelles
        previsions: DataFrame des prévisions
        
    Returns:
        Figure Plotly
    """
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=ventes_mensuelles.index,
        y=ventes_mensuelles['revenu_total'],
        mode='lines+markers',
        name='Revenu Réel',
        line=dict(color='rgb(55, 83, 109)', width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=ventes_mensuelles.index,
        y=ventes_mensuelles['revenu_moyen_mobile'],
        mode='lines',
        name='Moyenne Mobile (3 mois)',
        line=dict(color='orange', width=2, dash='dash')
    ))
    
    fig.add_trace(go.Scatter(
        x=previsions['date'],
        y=previsions['prevision'],
        mode='lines+markers',
        name='Prévision',
        line=dict(color='red', width=2, dash='dot')
    ))
    
    fig.update_layout(
        title='Évolution Temporelle du Revenu avec Prévisions',
        xaxis_title='Date',
        yaxis_title='Revenu Total ($)',
        template='plotly_white',
        height=400,
        hovermode='x unified'
    )
    
    return fig

In [13]:
def creer_graphique_spatial(distribution_spatiale: pd.DataFrame) -> go.Figure:
    """
    Crée un graphique de la distribution spatiale des ventes.
    
    Args:
        distribution_spatiale: DataFrame de la distribution par ville
        
    Returns:
        Figure Plotly
    """
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=distribution_spatiale['city'],
        y=distribution_spatiale['revenu_total'],
        marker=dict(
            color=distribution_spatiale['revenu_total'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='Revenu')
        ),
        text=distribution_spatiale['nombre_transactions'],
        texttemplate='%{text} trans.',
        textposition='outside'
    ))
    
    fig.update_layout(
        title='Top 20 Villes par Revenu Total',
        xaxis_title='Ville',
        yaxis_title='Revenu Total ($)',
        template='plotly_white',
        height=400,
        xaxis_tickangle=-45
    )
    
    return fig

---

## ÉTAPE 1 : Collecte et Exploration des Données

### Chargement des données

Le dataset **invoices.csv** contient des données synthétiques de facturation d'une boutique en ligne, générées avec la bibliothèque Python Faker. Il simule des transactions réalistes avec des informations client, produit et temporelles.

In [14]:
df_brut = charger_donnees('invoices.csv')
print(f"Données chargées avec succès: {df_brut.shape[0]} lignes et {df_brut.shape[1]} colonnes")
df_brut.head()

Données chargées avec succès: 10000 lignes et 11 colonnes


,first_name,last_name,email,product_id,qty,amount,invoice_date,address,city,stock_code,job
0,Carmen Nixon,Todd Anderson,marvinjackson@example.com,133,9,14.57,10/09/1982,283 Wendy Common,West Alexander,36239634,Logistics and distribution manager
1,Mrs. Heather Miller,Julia Moore,jeffrey84@example.net,155,5,65.48,03/10/2012,13567 Patricia Circles Apt. 751,Andreamouth,2820163,Osteopath
2,Crystal May,Philip Moody,ugoodman@example.com,151,9,24.66,23/03/1976,6389 Debbie Island Suite 470,Coxbury,27006726,Economist
3,Bobby Weber,Mark Scott,ssanchez@example.com,143,4,21.34,17/08/1986,6362 Ashley Plaza Apt. 994,Ninaland,83036521,Sports administrator
4,Kristen Welch,David David,cynthia66@example.net,168,2,83.90,11/06/1996,463 Steven Cliffs Suite 757,Isaiahview,80142652,Chief Marketing Officer


### Exploration détaillée

In [15]:
exploration = explorer_donnees(df_brut)

print("=" * 60)
print("RÉSUMÉ DE L'EXPLORATION DES DONNÉES")
print("=" * 60)
print(f"\nDimensions: {exploration['dimensions'][0]} lignes × {exploration['dimensions'][1]} colonnes")
print(f"\nColonnes disponibles: {', '.join(exploration['colonnes'])}")

RÉSUMÉ DE L'EXPLORATION DES DONNÉES

Dimensions: 10000 lignes × 11 colonnes

Colonnes disponibles: first_name, last_name, email, product_id, qty, amount, invoice_date, address, city, stock_code, job


In [16]:
print("\n" + "=" * 60)
print("TYPES DE DONNÉES")
print("=" * 60)
pd.DataFrame(exploration['types'].items(), columns=['Colonne', 'Type'])


TYPES DE DONNÉES


,Colonne,Type
0,first_name,object
1,last_name,object
2,email,object
3,product_id,int64
4,qty,int64
5,amount,float64
6,invoice_date,object
7,address,object
8,city,object
9,stock_code,int64


In [17]:
print("\n" + "=" * 60)
print("VALEURS MANQUANTES")
print("=" * 60)
valeurs_manquantes = pd.DataFrame(exploration['valeurs_manquantes'].items(), 
                                   columns=['Colonne', 'Nombre'])
print(f"\nTotal des valeurs manquantes: {valeurs_manquantes['Nombre'].sum()}")
print("\n✓ Aucune valeur manquante détectée dans le dataset" if valeurs_manquantes['Nombre'].sum() == 0 
      else valeurs_manquantes[valeurs_manquantes['Nombre'] > 0])


VALEURS MANQUANTES

Total des valeurs manquantes: 0

✓ Aucune valeur manquante détectée dans le dataset


In [18]:
print("\n" + "=" * 60)
print("STATISTIQUES DESCRIPTIVES")
print("=" * 60)
df_brut.describe()


STATISTIQUES DESCRIPTIVES


,product_id,qty,amount,stock_code
count,10000.000000,10000.000000,10000.000000,1.000000e+04
mean,149.746700,5.005900,52.918236,4.950036e+07
std,28.728186,2.576767,27.434579,2.903081e+07
min,100.000000,1.000000,5.010000,1.977000e+03
25%,125.000000,3.000000,29.137500,2.425234e+07
50%,150.000000,5.000000,53.485000,4.931714e+07
75%,175.000000,7.000000,76.520000,7.457446e+07
max,199.000000,9.000000,99.990000,9.999216e+07


In [19]:
print("\n" + "=" * 60)
print("INFORMATIONS COMPLÉMENTAIRES")
print("=" * 60)
print(f"\nNombre de produits uniques: {df_brut['product_id'].nunique()}")
print(f"Nombre de villes uniques: {df_brut['city'].nunique()}")
print(f"Nombre d'emails uniques: {df_brut['email'].nunique()}")
print(f"\nPlage de quantités: {df_brut['qty'].min()} - {df_brut['qty'].max()}")
print(f"Plage de montants: ${df_brut['amount'].min():.2f} - ${df_brut['amount'].max():.2f}")


INFORMATIONS COMPLÉMENTAIRES

Nombre de produits uniques: 100
Nombre de villes uniques: 7773
Nombre d'emails uniques: 9769

Plage de quantités: 1 - 9
Plage de montants: $5.01 - $99.99


### Préparation des données

Conversion des dates et création de colonnes dérivées pour faciliter les analyses temporelles et calculer le revenu total par transaction.

In [20]:
df = preparer_donnees(df_brut)
print("\n✓ Données préparées avec succès")
print(f"\nNouvelles colonnes créées: annee, mois, trimestre, jour_semaine, revenu_total")
print(f"\nPlage temporelle: {df['invoice_date'].min().date()} à {df['invoice_date'].max().date()}")
df.head()


✓ Données préparées avec succès

Nouvelles colonnes créées: annee, mois, trimestre, jour_semaine, revenu_total

Plage temporelle: 1970-01-05 à 2022-01-17


,first_name,last_name,email,product_id,qty,amount,invoice_date,address,city,stock_code,job,annee,mois,trimestre,jour_semaine,revenu_total
0,Carmen Nixon,Todd Anderson,marvinjackson@example.com,133,9,14.57,1982-09-10,283 Wendy Common,West Alexander,36239634,Logistics and distribution manager,1982,9,3,4,131.13
1,Mrs. Heather Miller,Julia Moore,jeffrey84@example.net,155,5,65.48,2012-10-03,13567 Patricia Circles Apt. 751,Andreamouth,2820163,Osteopath,2012,10,4,2,327.40
2,Crystal May,Philip Moody,ugoodman@example.com,151,9,24.66,1976-03-23,6389 Debbie Island Suite 470,Coxbury,27006726,Economist,1976,3,1,1,221.94
3,Bobby Weber,Mark Scott,ssanchez@example.com,143,4,21.34,1986-08-17,6362 Ashley Plaza Apt. 994,Ninaland,83036521,Sports administrator,1986,8,3,6,85.36
4,Kristen Welch,David David,cynthia66@example.net,168,2,83.90,1996-06-11,463 Steven Cliffs Suite 757,Isaiahview,80142652,Chief Marketing Officer,1996,6,2,1,167.80


---

## ÉTAPE 2 : Construction des Indicateurs

In [21]:
def analyser_cycle_vie_produits(df: pd.DataFrame) -> pd.DataFrame:
    """
    INDICATEUR 5 - Analyse du cycle de vie:
    Identifie la phase de chaque produit (croissance/maturité/déclin)
    """
    evolution = df.groupby(['product_id', df['invoice_date'].dt.to_period('Q')]).agg({
        'revenu_total': 'sum'
    }).reset_index()
    
    resultats = []
    for produit in evolution['product_id'].unique():
        data_produit = evolution[evolution['product_id'] == produit].sort_values('invoice_date')
        
        if len(data_produit) >= 3:
            ventes = data_produit['revenu_total'].values
            x = np.arange(len(ventes))
            tendance = np.polyfit(x, ventes, 1)[0]
            
            if tendance > 100:
                phase = 'Croissance'
            elif tendance < -100:
                phase = 'Déclin'
            else:
                phase = 'Maturité'
                
            resultats.append({
                'product_id': produit,
                'phase': phase,
                'tendance': tendance,
                'ventes_premiere_periode': ventes[0],
                'ventes_derniere_periode': ventes[-1],
                'variation_pct': ((ventes[-1] - ventes[0]) / ventes[0] * 100)
            })
    
    return pd.DataFrame(resultats)

In [22]:
def analyser_saisonnalite_avancee(df: pd.DataFrame) -> dict:
    """
    Détecte les patterns temporels par jour, mois, trimestre
    """
    analyses = {
        'par_mois': df.groupby('mois')['revenu_total'].sum().to_dict(),
        'par_jour_semaine': df.groupby('jour_semaine')['revenu_total'].mean().to_dict(),
        'par_trimestre': df.groupby('trimestre')['revenu_total'].sum().to_dict()
    }
    
    # Meilleur mois
    meilleur_mois = max(analyses['par_mois'], key=analyses['par_mois'].get)
    
    # Meilleur jour
    meilleur_jour = max(analyses['par_jour_semaine'], key=analyses['par_jour_semaine'].get)
    jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    
    return {
        'analyses': analyses,
        'insights': {
            'meilleur_mois': meilleur_mois,
            'meilleur_jour': jours[meilleur_jour]
        }
    }

In [23]:
def segmenter_clients(df: pd.DataFrame) -> pd.DataFrame:
    """
    Segmentation RFM simplifiée des clients
    """
    derniere_date = df['invoice_date'].max()
    
    rfm = df.groupby('email').agg({
        'invoice_date': lambda x: (derniere_date - x.max()).days,  # Recency
        'product_id': 'count',  # Frequency
        'revenu_total': 'sum'  # Monetary
    }).rename(columns={
        'invoice_date': 'recency',
        'product_id': 'frequency',
        'revenu_total': 'monetary'
    })
    
    # Scores (1-4)
    rfm['r_score'] = pd.qcut(rfm['recency'], 4, labels=[4,3,2,1])
    rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 4, labels=[1,2,3,4])
    rfm['m_score'] = pd.qcut(rfm['monetary'], 4, labels=[1,2,3,4])
    
    rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)
    
    # Segmentation
    def segment_client(score):
        if score[0] in ['3', '4'] and score[1] in ['3', '4']:
            return 'VIP'
        elif score[0] in ['1', '2']:
            return 'À risque'
        elif score[1] in ['1', '2']:
            return 'Nouveau'
        else:
            return 'Régulier'
    
    rfm['segment'] = rfm['rfm_score'].apply(segment_client)
    
    return rfm.reset_index()

### INDICATEUR 1 : Top 10 Produits par Revenu Total (Requête de groupement)

**Description**: Cette analyse identifie les 10 produits les plus rentables en calculant le revenu total généré par chaque produit (quantité × montant). Elle permet de comprendre quels produits génèrent le plus de revenus pour l'entreprise.

**Méthode**: Utilisation de `groupby()` sur `product_id` avec agrégation de plusieurs métriques (somme des revenus, somme des quantités, nombre de transactions). Le calcul du prix moyen par produit permet d'identifier si un produit est rentable par son volume ou par son prix élevé.

**Interprétation**: Les produits en tête de classement représentent les sources principales de revenus et méritent une attention particulière en termes de stock et de marketing.

In [24]:
top_produits = analyser_top_produits(df, n=10)
print("=" * 70)
print("TOP 10 PRODUITS PAR REVENU TOTAL")
print("=" * 70)
top_produits

TOP 10 PRODUITS PAR REVENU TOTAL


,product_id,revenu_total,qty,nombre_transactions,prix_moyen
0,164,41837.15,775,131,53.983419
1,128,35945.10,631,119,56.965293
2,108,35076.94,605,112,57.978413
3,143,33040.73,626,114,52.780719
4,145,32726.90,628,126,52.112898
5,195,32258.20,580,107,55.617586
6,159,31568.54,578,109,54.616851
7,104,31383.09,584,118,53.738168
8,112,30725.67,570,107,53.904684
9,186,30617.91,521,103,58.767582


In [25]:
print(f"\n📊 INSIGHTS:")
print(f"  • Produit le plus rentable: ID {top_produits.iloc[0]['product_id']} (${top_produits.iloc[0]['revenu_total']:.2f})")
print(f"  • Revenu total des top 10: ${top_produits['revenu_total'].sum():.2f}")
print(f"  • Part du revenu total: {(top_produits['revenu_total'].sum() / df['revenu_total'].sum() * 100):.2f}%")


📊 INSIGHTS:
  • Produit le plus rentable: ID 164.0 ($41837.15)
  • Revenu total des top 10: $335180.23
  • Part du revenu total: 12.62%


---

### INDICATEUR 2 : Normalisation et Discrétisation des Montants (Transformation de données)

**Description**: Cette transformation applique deux techniques de prétraitement des données : (1) la normalisation min-max qui ramène les valeurs entre 0 et 1, facilitant la comparaison entre variables d'échelles différentes, et (2) la discrétisation qui catégorise les montants continus en classes ordinales.

**Méthode**: La normalisation min-max utilise la formule `(x - min) / (max - min)`. La discrétisation avec `pd.cut()` crée 4 catégories basées sur des seuils de montants (0-25, 25-50, 50-75, 75-100). Ces transformations sont essentielles pour le machine learning et l'analyse exploratoire.

**Interprétation**: La distribution des catégories révèle les segments de prix prédominants. Une forte concentration dans les catégories "Faible" ou "Moyen" indique un positionnement économique, tandis qu'une distribution équilibrée suggère une gamme de produits variée.

In [26]:
df_normalise = normaliser_montants(df, methode='minmax')
print("=" * 70)
print("TRANSFORMATION DES DONNÉES - NORMALISATION ET DISCRÉTISATION")
print("=" * 70)
print("\n📊 Statistiques des valeurs normalisées:")
df_normalise[['montant_normalise', 'quantite_normalisee']].describe()

TRANSFORMATION DES DONNÉES - NORMALISATION ET DISCRÉTISATION

📊 Statistiques des valeurs normalisées:


,montant_normalise,quantite_normalisee
count,10000.000000,10000.000000
mean,0.504403,0.500737
std,0.288846,0.322096
min,0.000000,0.000000
25%,0.254027,0.250000
50%,0.510371,0.500000
75%,0.752895,0.750000
max,1.000000,1.000000


In [27]:
print("\n📊 Distribution des catégories de montants:")
distribution_cat = df_normalise['categorie_montant'].value_counts().sort_index()
for categorie, count in distribution_cat.items():
    pourcentage = (count / len(df_normalise)) * 100
    print(f"  • {categorie}: {count} transactions ({pourcentage:.2f}%)")


📊 Distribution des catégories de montants:
  • Faible: 2072 transactions (20.72%)
  • Moyen: 2570 transactions (25.70%)
  • Élevé: 2677 transactions (26.77%)
  • Très élevé: 2681 transactions (26.81%)


---

### INDICATEUR 3 : Analyse Temporelle avec Prévisions (Analyse temporelle)

**Description**: Cette analyse examine l'évolution des ventes dans le temps en agrégeant les données par mois. Elle calcule également une moyenne mobile sur 3 mois pour lisser les variations et révéler les tendances sous-jacentes. Une prévision simple est ensuite générée pour anticiper les ventes futures.

**Méthode**: Utilisation de `pd.Period` pour l'agrégation mensuelle, puis calcul d'une moyenne mobile avec `rolling(window=3)`. La prévision utilise la moyenne des 6 derniers mois comme estimation simple. Cette approche pourrait être améliorée avec des modèles ARIMA ou Prophet pour des prévisions plus sophistiquées.

**Interprétation**: Les pics et creux révèlent la saisonnalité des ventes. Une tendance haussière de la moyenne mobile indique une croissance, tandis qu'une tendance baissière suggère un déclin nécessitant des actions correctives. Les prévisions guident la planification des stocks et des campagnes marketing.

In [28]:
ventes_mensuelles = analyser_tendances_temporelles(df_normalise)
print("=" * 70)
print("ANALYSE TEMPORELLE - ÉVOLUTION DES VENTES MENSUELLES")
print("=" * 70)
ventes_mensuelles.head(10)

ANALYSE TEMPORELLE - ÉVOLUTION DES VENTES MENSUELLES


,revenu_total,qty,nombre_transactions,revenu_moyen_mobile
invoice_date,,,,
1970-01-01,3878.93,87,15,3878.930000
1970-02-01,6050.15,107,18,4964.540000
1970-03-01,5527.01,96,21,5152.030000
1970-04-01,2050.53,32,10,4542.563333
1970-05-01,2779.05,57,9,3452.196667
1970-06-01,4441.93,78,17,3090.503333
1970-07-01,5632.33,91,17,4284.436667
1970-08-01,3741.50,75,17,4605.253333
1970-09-01,3159.37,71,14,4177.733333


In [29]:
print(f"\n📊 STATISTIQUES TEMPORELLES:")
print(f"  • Période couverte: {ventes_mensuelles.index.min().date()} à {ventes_mensuelles.index.max().date()}")
print(f"  • Nombre de mois: {len(ventes_mensuelles)}")
print(f"  • Revenu mensuel moyen: ${ventes_mensuelles['revenu_total'].mean():.2f}")
print(f"  • Meilleur mois: {ventes_mensuelles['revenu_total'].idxmax().date()} (${ventes_mensuelles['revenu_total'].max():.2f})")
print(f"  • Pire mois: {ventes_mensuelles['revenu_total'].idxmin().date()} (${ventes_mensuelles['revenu_total'].min():.2f})")


📊 STATISTIQUES TEMPORELLES:
  • Période couverte: 1970-01-01 à 2022-01-01
  • Nombre de mois: 625
  • Revenu mensuel moyen: $4250.99
  • Meilleur mois: 1989-11-01 ($9452.02)
  • Pire mois: 2011-11-01 ($1037.91)


In [30]:
previsions = prevoir_ventes(ventes_mensuelles, periodes_futures=12)
print("\n📈 PRÉVISIONS POUR LES 12 PROCHAINS MOIS:")
print(f"  • Revenu mensuel prévu: ${previsions['prevision'].iloc[0]:.2f}")
print(f"  • Revenu annuel estimé: ${previsions['prevision'].iloc[0] * 12:.2f}")
previsions.head()


📈 PRÉVISIONS POUR LES 12 PROCHAINS MOIS:
  • Revenu mensuel prévu: $4058.75
  • Revenu annuel estimé: $48704.94


,date,prevision
0,2022-02-01,4058.745
1,2022-03-01,4058.745
2,2022-04-01,4058.745
3,2022-05-01,4058.745
4,2022-06-01,4058.745


---

### INDICATEUR 4 : Distribution Géographique des Ventes (Analyse spatiale)

**Description**: Cette analyse spatiale examine comment les ventes sont distribuées géographiquement en agrégeant les données par ville. Elle révèle les marchés les plus lucratifs et permet d'identifier les opportunités d'expansion ou les zones nécessitant plus d'efforts marketing.

**Méthode**: Groupement par ville avec calcul de plusieurs métriques : revenu total, revenu moyen par transaction, nombre de transactions, quantité totale vendue et montant moyen. Le classement des top 20 villes permet de concentrer l'analyse sur les marchés principaux.

**Interprétation**: Les villes en tête de classement représentent les marchés clés où l'entreprise a une forte présence. Un revenu moyen élevé mais peu de transactions suggère des clients premium, tandis qu'un grand nombre de transactions avec un montant moyen faible indique un marché de volume. Ces insights orientent la stratégie de développement géographique.

In [31]:
distribution_spatiale = analyser_distribution_spatiale(df_normalise)
print("=" * 70)
print("ANALYSE SPATIALE - DISTRIBUTION PAR VILLE")
print("=" * 70)
distribution_spatiale

ANALYSE SPATIALE - DISTRIBUTION PAR VILLE


,city,revenu_total,revenu_moyen,nombre_transactions,quantite_totale,montant_moyen
0,Lake James,4417.17,368.097500,12,65,64.869167
1,Port Kimberly,3144.10,524.016667,6,38,76.626667
2,North Michael,2902.27,362.783750,8,50,58.415000
3,Smithmouth,2644.53,330.566250,8,41,58.706250
4,South Jennifer,2570.33,285.592222,9,47,53.977778
5,Port Joshua,2566.84,366.691429,7,41,57.874286
6,South James,2504.30,250.430000,10,55,40.392000
7,Jamesstad,2381.87,476.374000,5,33,73.708000
8,Lake Michael,2317.09,257.454444,9,50,46.927778
9,South David,2309.46,256.606667,9,57,34.834444


In [32]:
print(f"\n📊 INSIGHTS GÉOGRAPHIQUES:")
print(f"  • Ville la plus rentable: {distribution_spatiale.iloc[0]['city']} (${distribution_spatiale.iloc[0]['revenu_total']:.2f})")
print(f"  • Revenu moyen le plus élevé: {distribution_spatiale.loc[distribution_spatiale['revenu_moyen'].idxmax(), 'city']} (${distribution_spatiale['revenu_moyen'].max():.2f})")
print(f"  • Ville avec le plus de transactions: {distribution_spatiale.loc[distribution_spatiale['nombre_transactions'].idxmax(), 'city']} ({int(distribution_spatiale['nombre_transactions'].max())} trans.)")
print(f"  • Concentration: Top 5 villes = {(distribution_spatiale.head()['revenu_total'].sum() / df['revenu_total'].sum() * 100):.2f}% du revenu total")


📊 INSIGHTS GÉOGRAPHIQUES:
  • Ville la plus rentable: Lake James ($4417.17)
  • Revenu moyen le plus élevé: Johnmouth ($526.75)
  • Ville avec le plus de transactions: Lake James (12 trans.)
  • Concentration: Top 5 villes = 0.59% du revenu total


## INDICATEUR 5 : Cycle de Vie des Produits (Analyse d'évolution temporelle)

**Description**: Cette analyse identifie la phase de vie de chaque produit en calculant la tendance de ses ventes sur les trimestres. Elle permet de classifier les produits en croissance, maturité ou déclin.

**Méthode**: Régression linéaire simple sur les ventes trimestrielles de chaque produit. Une pente positive indique une croissance, négative un déclin, proche de zéro une maturité.

**Interprétation**: Les produits en croissance méritent plus d'investissement marketing. Les produits en déclin nécessitent soit une relance, soit un arrêt progressif.

In [33]:
cycle_vie = analyser_cycle_vie_produits(df_normalise)
print("=" * 70)
print("CYCLE DE VIE DES PRODUITS")
print("=" * 70)
cycle_vie.sort_values('tendance', ascending=False).head(10)

CYCLE DE VIE DES PRODUITS


,product_id,phase,tendance,ventes_premiere_periode,ventes_derniere_periode,variation_pct
69,169,Maturité,3.097516,23.56,665.92,2726.485569
55,155,Maturité,2.755487,26.60,654.75,2361.466165
68,168,Maturité,2.472177,94.04,9.83,-89.547001
54,154,Maturité,2.432944,126.08,203.76,61.611675
43,143,Maturité,2.300855,390.98,370.98,-5.115351
81,181,Maturité,2.210142,434.49,129.54,-70.185735
71,171,Maturité,2.048042,42.43,304.90,618.595333
36,136,Maturité,2.034907,43.68,345.92,691.941392
59,159,Maturité,2.014690,287.70,394.25,37.035106
9,109,Maturité,1.950708,401.15,66.60,-83.397732


In [34]:
print(f"\n📊 INSIGHTS CYCLE DE VIE:")
print(f"  • Produits en croissance: {len(cycle_vie[cycle_vie['phase'] == 'Croissance'])}")
print(f"  • Produits matures: {len(cycle_vie[cycle_vie['phase'] == 'Maturité'])}")
print(f"  • Produits en déclin: {len(cycle_vie[cycle_vie['phase'] == 'Déclin'])}")
print(f"\n  • Top produit en croissance: ID {cycle_vie.loc[cycle_vie['tendance'].idxmax(), 'product_id']} (+{cycle_vie['tendance'].max():.2f}/trimestre)")


📊 INSIGHTS CYCLE DE VIE:
  • Produits en croissance: 0
  • Produits matures: 100
  • Produits en déclin: 0

  • Top produit en croissance: ID 169 (+3.10/trimestre)


**Description**: Analyse les patterns de vente selon différentes dimensions temporelles (mois, jour de la semaine, trimestre) pour identifier les périodes clés.

**Méthode**: Agrégation des revenus par période temporelle avec identification des pics et creux de vente.

**Interprétation**: Permet d'optimiser les campagnes marketing et la gestion des stocks en fonction des périodes de forte/faible demande.

In [35]:
saisonnalite = analyser_saisonnalite_avancee(df_normalise)
print("=" * 70)
print("ANALYSE DE SAISONNALITÉ")
print("=" * 70)
print(f"\n📅 MEILLEUR MOIS: {saisonnalite['insights']['meilleur_mois']}")
print(f"📅 MEILLEUR JOUR: {saisonnalite['insights']['meilleur_jour']}")

print("\n💰 Revenus par mois:")
for mois, revenu in sorted(saisonnalite['analyses']['par_mois'].items()):
    mois_nom = ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun', 'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc'][mois-1]
    print(f"  • {mois_nom}: ${revenu:.2f}")

ANALYSE DE SAISONNALITÉ

📅 MEILLEUR MOIS: 10
📅 MEILLEUR JOUR: Mercredi

💰 Revenus par mois:
  • Jan: $210288.42
  • Fév: $203505.54
  • Mar: $235716.39
  • Avr: $208591.37
  • Mai: $233641.77
  • Jun: $232542.47
  • Jul: $222428.48
  • Aoû: $219436.44
  • Sep: $209256.79
  • Oct: $237549.37
  • Nov: $226217.34
  • Déc: $217696.00


## INDICATEUR 7 : Segmentation Clients (RFM)

**Description**: Segmente les clients selon trois critères : Récence (dernière visite), Fréquence (nombre d'achats) et Montant (valeur totale). Identifie les clients VIP, réguliers, nouveaux et à risque.

**Méthode**: Analyse RFM avec scores quartiles (1-4) combinés pour créer un score global. Classification automatique en segments business.

**Interprétation**: Permet des stratégies marketing ciblées : fidélisation VIP, réactivation clients à risque, conversion nouveaux clients.

In [36]:
segments_clients = segmenter_clients(df_normalise)
print("=" * 70)
print("SEGMENTATION CLIENTS (RFM)")
print("=" * 70)
print("\n📊 Distribution des segments:")
print(segments_clients['segment'].value_counts())

SEGMENTATION CLIENTS (RFM)

📊 Distribution des segments:
segment
À risque    4884
VIP         2450
Nouveau     2435
Name: count, dtype: int64


In [37]:
print(f"\n💎 INSIGHTS CLIENTS:")
for segment in ['VIP', 'Régulier', 'Nouveau', 'À risque']:
    data_segment = segments_clients[segments_clients['segment'] == segment]
    if len(data_segment) > 0:
        print(f"\n  • {segment}:")
        print(f"    - Nombre: {len(data_segment)}")
        print(f"    - Revenu moyen: ${data_segment['monetary'].mean():.2f}")
        print(f"    - Fréquence moyenne: {data_segment['frequency'].mean():.1f} achats")


💎 INSIGHTS CLIENTS:

  • VIP:
    - Nombre: 2450
    - Revenu moyen: $280.78
    - Fréquence moyenne: 1.1 achats

  • Nouveau:
    - Nombre: 2435
    - Revenu moyen: $268.71
    - Fréquence moyenne: 1.0 achats

  • À risque:
    - Nombre: 4884
    - Revenu moyen: $269.18
    - Fréquence moyenne: 1.0 achats


---

## ÉTAPE 3 : Dashboard de Visualisation

### Création du Dashboard Interactif avec Dash

Le dashboard ci-dessous présente les 4 indicateurs sous forme de visualisations interactives. Chaque graphique peut être exploré en détail (zoom, survol, etc.).

In [38]:
def creer_dashboard(top_produits, df_normalise, ventes_mensuelles, previsions, distribution_spatiale):
    """
    Crée et lance le dashboard Dash avec les 4 indicateurs.
    
    Args:
        top_produits: DataFrame des meilleurs produits
        df_normalise: DataFrame avec données normalisées
        ventes_mensuelles: DataFrame des ventes mensuelles
        previsions: DataFrame des prévisions
        distribution_spatiale: DataFrame de la distribution géographique
    """
    app = dash.Dash(__name__)
    
    app.layout = html.Div([
        html.Div([
            html.H1("Dashboard d'Analyse des Factures", 
                    style={'textAlign': 'center', 'color': '#2c3e50', 'marginBottom': 10}),
            html.H3("Dataset: invoices.csv", 
                    style={'textAlign': 'center', 'color': '#7f8c8d', 'marginTop': 0}),
            html.Div([
                html.P("Membres de l'équipe: Prénom NOM (Responsable), Prénom NOM, Prénom NOM, Prénom NOM",
                       style={'textAlign': 'center', 'color': '#95a5a6'})
            ]),
            html.Div([
                html.H4("Objectif du Projet", style={'color': '#34495e'}),
                html.P("Analyser les données de facturation pour extraire 4 indicateurs clés permettant de comprendre "
                       "les tendances temporelles des ventes, la distribution géographique des clients, les produits "
                       "les plus rentables et les patterns de consommation.",
                       style={'color': '#7f8c8d', 'textAlign': 'justify'})
            ], style={'backgroundColor': '#ecf0f1', 'padding': '20px', 'borderRadius': '10px', 'margin': '20px'})
        ]),
        
        html.Div([
            html.Div([
                html.H3("Indicateur 1: Top Produits", style={'color': '#2980b9'}),
                dcc.Graph(figure=creer_graphique_top_produits(top_produits))
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),
            
            html.Div([
                html.H3("Indicateur 2: Distribution des Catégories", style={'color': '#27ae60'}),
                dcc.Graph(figure=creer_graphique_normalisation(df_normalise))
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        ]),
        
        html.Div([
            html.H3("Indicateur 3: Tendances Temporelles et Prévisions", style={'color': '#e67e22'}),
            dcc.Graph(figure=creer_graphique_tendances(ventes_mensuelles, previsions))
        ], style={'padding': '10px'}),
        
        html.Div([
            html.H3("Indicateur 4: Distribution Géographique", style={'color': '#8e44ad'}),
            dcc.Graph(figure=creer_graphique_spatial(distribution_spatiale))
        ], style={'padding': '10px'})
    ])
    
    return app

---

## Fonction Principale (Main)

In [39]:
def main():
    """
    Fonction principale orchestrant l'ensemble du projet.
    Exécute toutes les étapes: chargement, exploration, analyse et visualisation.
    """
    print("\n" + "="*70)
    print("DÉMARRAGE DU PROJET D'ANALYSE DES FACTURES")
    print("="*70 + "\n")
    
    chemin_fichier = 'invoices.csv'
    
    print("[1/6] Chargement des données...")
    df_brut = charger_donnees(chemin_fichier)
    print(f"      ✓ {df_brut.shape[0]} lignes chargées\n")
    
    print("[2/6] Exploration des données...")
    exploration = explorer_donnees(df_brut)
    print(f"      ✓ {len(exploration['colonnes'])} colonnes analysées\n")
    
    print("[3/6] Préparation des données...")
    df = preparer_donnees(df_brut)
    print("      ✓ Dates converties et colonnes dérivées créées\n")
    
    print("[4/6] Construction des indicateurs...")
    top_produits = analyser_top_produits(df, n=10)
    print("      ✓ Indicateur 1: Top produits calculé")
    
    df_normalise = normaliser_montants(df, methode='minmax')
    print("      ✓ Indicateur 2: Normalisation effectuée")
    
    ventes_mensuelles = analyser_tendances_temporelles(df_normalise)
    previsions = prevoir_ventes(ventes_mensuelles, periodes_futures=12)
    print("      ✓ Indicateur 3: Analyse temporelle complétée")
    
    distribution_spatiale = analyser_distribution_spatiale(df_normalise)
    print("      ✓ Indicateur 4: Distribution spatiale analysée\n")
    
    print("[5/6] Création des visualisations...")
    fig1 = creer_graphique_top_produits(top_produits)
    fig2 = creer_graphique_normalisation(df_normalise)
    fig3 = creer_graphique_tendances(ventes_mensuelles, previsions)
    fig4 = creer_graphique_spatial(distribution_spatiale)
    print("      ✓ 4 graphiques créés\n")
    
    print("[6/6] Lancement du dashboard...")
    app = creer_dashboard(top_produits, df_normalise, ventes_mensuelles, 
                          previsions, distribution_spatiale)
    print("      ✓ Dashboard prêt\n")
    
    print("="*70)
    print("PROJET TERMINÉ AVEC SUCCÈS")
    print("="*70)
    print("\n💡 Pour lancer le dashboard, exécutez: app.run_server(debug=True, port=8050)")
    print("   Puis ouvrez votre navigateur à l'adresse: http://127.0.0.1:8050/\n")
    
    return app

### Exécution du programme principal

In [40]:
if __name__ == "__main__":
    app = main()
    app.run(debug=True, port=8050)


DÉMARRAGE DU PROJET D'ANALYSE DES FACTURES

[1/6] Chargement des données...
      ✓ 10000 lignes chargées

[2/6] Exploration des données...
      ✓ 11 colonnes analysées

[3/6] Préparation des données...
      ✓ Dates converties et colonnes dérivées créées

[4/6] Construction des indicateurs...
      ✓ Indicateur 1: Top produits calculé
      ✓ Indicateur 2: Normalisation effectuée
      ✓ Indicateur 3: Analyse temporelle complétée
      ✓ Indicateur 4: Distribution spatiale analysée

[5/6] Création des visualisations...
      ✓ 4 graphiques créés

[6/6] Lancement du dashboard...
      ✓ Dashboard prêt

PROJET TERMINÉ AVEC SUCCÈS

💡 Pour lancer le dashboard, exécutez: app.run_server(debug=True, port=8050)
   Puis ouvrez votre navigateur à l'adresse: http://127.0.0.1:8050/



OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

---

## Recommandations Pédagogiques

### Points Clés de la Solution

1. **Organisation modulaire**: Chaque fonctionnalité est encapsulée dans une fonction dédiée avec docstring complète
2. **Respect des étapes KDD**: Collection → Préparation → Analyse → Visualisation
3. **4 indicateurs diversifiés**: 
   - Requête de groupement (top produits)
   - Transformation de données (normalisation + discrétisation)
   - Analyse temporelle avec prévisions
   - Analyse spatiale (distribution géographique)
4. **Dashboard interactif**: Interface professionnelle avec Plotly Dash
5. **Code production-ready**: Typage, gestion d'erreurs, performance optimisée

### Pièges Courants à Éviter

1. **Dates mal converties**: Toujours vérifier le format avec `pd.to_datetime()` et spécifier le format
2. **Agrégations incorrectes**: Bien comprendre la différence entre `sum()`, `mean()`, `count()`
3. **Visualisations surchargées**: Privilégier la clarté à la complexité
4. **Absence de validation**: Toujours afficher `.head()`, `.info()` après transformation
5. **Dashboard qui ne se lance pas**: Vérifier les ports et les dépendances Dash

### Concepts Importants Illustrés

1. **Agrégation et groupement**: `groupby()` avec multiples fonctions d'agrégation
2. **Séries temporelles**: Manipulation de dates, périodes, moyennes mobiles
3. **Normalisation**: Min-max scaling pour comparer des variables d'échelles différentes
4. **Discrétisation**: Conversion de variables continues en catégories ordinales
5. **Visualisation interactive**: Utilisation de Plotly et Dash pour créer des dashboards

### Suggestions d'Améliorations Possibles

1. **Dataset externe**: Enrichir avec des données démographiques par ville ou des indices économiques
2. **Prévisions avancées**: Implémenter ARIMA, Prophet ou LSTM pour des prévisions plus précises
3. **Clustering spatial**: Utiliser K-means ou DBSCAN pour segmenter les villes
4. **Analyse de panier**: Implémenter l'algorithme Apriori pour trouver des associations de produits
5. **Filtres interactifs**: Ajouter des dropdowns dans le dashboard pour filtrer par période ou ville
6. **Détection d'anomalies**: Identifier les transactions inhabituelles avec Isolation Forest
7. **Tests unitaires**: Ajouter des tests avec pytest pour valider chaque fonction
8. **Export des résultats**: Permettre le téléchargement des analyses en CSV ou Excel